# 1. Importing modules and functions

In [4]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from molvs import standardize_smiles
import numpy as np
import pandas as pd
from copy import deepcopy
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split,KFold, StratifiedKFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import joblib
import pickle
from numpy import savetxt
from IPython.display import HTML
from rdkit.Chem import PandasTools

In [5]:
def convert_smi_to_canon_smi(smi):
    
    try:
        canon_smi = Chem.MolToSmiles(Chem.MolFromSmiles(smi),isomericSmiles = False)
    except:
        canon_smi='wrong_smiles'
    return canon_smi

In [6]:
def standart(smi):
    global m
    if smi!='wrong_smiles':
        try:
            smiles=standardize_smiles(smi)
            m = Chem.MolFromSmiles(smi)
        except:
            smi='error kekule'
    else:
        m = 'check the smiles'
    return m

# MORGAN FP_Gradient Boosting Rat oral LD50, mg/kg  

## Load data and curation work set

In [7]:
# Set file path
df_ws=pd.read_csv('rat_oral_LD50_WS.csv')
df_ws

,CAS_Number,SMILES,pLD50,"LD50,mg/kg"
0,626-48-2,Cc1cc(=O)[nH]c(=O)[nH]1,0.291207,64463.0000
1,27849-94-1,CC(CCc1ccc2c(c1)OCO2)NN,0.440660,75449.0000
2,110-54-3,CCCCCC,0.537460,24980.0000
3,1330-92-3,CCCCC(CC)COC(=O)C1=C(C(=O)OCC(CC)CCCC)CCCC1,0.539250,113917.0000
4,57-55-6,CC(O)CO,0.580330,19989.0000
...,...,...,...,...
7707,56073-10-0,O=c1oc2ccccc2c(O)c1C1CC(c2ccc(-c3ccc(Br)cc3)cc...,6.514700,0.1596
7708,130209-82-4,CC(C)OC(=O)CCC/C=C/CC1C(O)CC(O)C1CCC(O)CCc1ccccc1,6.937100,0.0500
7709,83805-11-2,C=C1/C(=C\C=C2/CCCC3(C)C2CCC3C(C)CCCC(O)(C(F)(...,7.099700,0.0417
7710,1746-01-6,Clc1cc2c(cc1Cl)Oc1cc(Cl)c(Cl)cc1O2,7.206800,0.0199


 Convert a SMILES string to canonical SMILES

In [8]:
df_ws1 = deepcopy(df_ws)
df_ws1["SMILES"] = df_ws1.apply(lambda x: convert_smi_to_canon_smi(x.SMILES), axis=1)
df_ws1

,CAS_Number,SMILES,pLD50,"LD50,mg/kg"
0,626-48-2,Cc1cc(=O)[nH]c(=O)[nH]1,0.291207,64463.0000
1,27849-94-1,CC(CCc1ccc2c(c1)OCO2)NN,0.440660,75449.0000
2,110-54-3,CCCCCC,0.537460,24980.0000
3,1330-92-3,CCCCC(CC)COC(=O)C1=C(C(=O)OCC(CC)CCCC)CCCC1,0.539250,113917.0000
4,57-55-6,CC(O)CO,0.580330,19989.0000
...,...,...,...,...
7707,56073-10-0,O=c1oc2ccccc2c(O)c1C1CC(c2ccc(-c3ccc(Br)cc3)cc...,6.514700,0.1596
7708,130209-82-4,CC(C)OC(=O)CCCC=CCC1C(O)CC(O)C1CCC(O)CCc1ccccc1,6.937100,0.0500
7709,83805-11-2,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CCCC(O)(C(F)(F)F...,7.099700,0.0417
7710,1746-01-6,Clc1cc2c(cc1Cl)Oc1cc(Cl)c(Cl)cc1O2,7.206800,0.0199


In [9]:
print('Original data: ', len(df_ws), 'molecules')
print('Failed data: ', len(df_ws1[df_ws1['SMILES']=='wrong_smiles']), 'molecules')

Original data:  7712 molecules
Failed data:  0 molecules


In [10]:
index=df_ws1.index[df_ws1['SMILES']=='wrong_smiles'].tolist()
wrong_smiles=df_ws.iloc[index]
wrong_smiles=wrong_smiles.SMILES
number=[x+1 for x in index]
bad_molecules = pd.DataFrame({'No. failed smiles in original set': number, 'SMILES of wrong structure: ': wrong_smiles}, index=None)
bad_molecules = bad_molecules.set_index('No. failed smiles in original set')
bad_molecules

,SMILES of wrong structure:
No. failed smiles in original set,


##  Standardization  for work set

In [11]:
df_ws1["Molecule"] = df_ws1.apply(lambda x: standart(x.SMILES), axis=1)
moldf_ws=df_ws1[df_ws1['SMILES']!='wrong_smiles']
print('Kept data: ', len(moldf_ws), 'molecules')

Kept data:  7712 molecules


In [12]:
moldf_ws

,CAS_Number,SMILES,pLD50,"LD50,mg/kg",Molecule
0,626-48-2,Cc1cc(=O)[nH]c(=O)[nH]1,0.291207,64463.0000,<rdkit.Chem.rdchem.Mol object at 0x000001C71BA...
1,27849-94-1,CC(CCc1ccc2c(c1)OCO2)NN,0.440660,75449.0000,<rdkit.Chem.rdchem.Mol object at 0x000001C71BA...
2,110-54-3,CCCCCC,0.537460,24980.0000,<rdkit.Chem.rdchem.Mol object at 0x000001C71BA...
3,1330-92-3,CCCCC(CC)COC(=O)C1=C(C(=O)OCC(CC)CCCC)CCCC1,0.539250,113917.0000,<rdkit.Chem.rdchem.Mol object at 0x000001C71BA...
4,57-55-6,CC(O)CO,0.580330,19989.0000,<rdkit.Chem.rdchem.Mol object at 0x000001C71BA...
...,...,...,...,...,...
7707,56073-10-0,O=c1oc2ccccc2c(O)c1C1CC(c2ccc(-c3ccc(Br)cc3)cc...,6.514700,0.1596,<rdkit.Chem.rdchem.Mol object at 0x000001C71FC...
7708,130209-82-4,CC(C)OC(=O)CCCC=CCC1C(O)CC(O)C1CCC(O)CCc1ccccc1,6.937100,0.0500,<rdkit.Chem.rdchem.Mol object at 0x000001C71FC...
7709,83805-11-2,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CCCC(O)(C(F)(F)F...,7.099700,0.0417,<rdkit.Chem.rdchem.Mol object at 0x000001C71FC...
7710,1746-01-6,Clc1cc2c(cc1Cl)Oc1cc(Cl)c(Cl)cc1O2,7.206800,0.0199,<rdkit.Chem.rdchem.Mol object at 0x000001C71FC...


In [13]:
y_tr=moldf_ws.pLD50
y_tr

0       0.291207
1       0.440660
2       0.537460
3       0.539250
4       0.580330
          ...   
7707    6.514700
7708    6.937100
7709    7.099700
7710    7.206800
7711    7.602600
Name: pLD50, Length: 7712, dtype: float64

In [14]:
moldf_ws=moldf_ws.Molecule

##  Load data and curation test set

In [15]:
df_ts=pd.read_csv('rat_oral_LD50_TS.csv')
df_ts

,CAS_Number,SMILES,pLD50,"LD50,mg/kg"
0,7782-40-3,C,0.017765,15388.8000
1,2842-38-8,OCCNC1CCCCC1,0.572840,38274.0000
2,66257-53-2,NC(=O)C(=O)O,0.624490,21133.0000
3,2173-56-0,CCCCCOC(=O)CCCC,0.686960,35395.0000
4,4726-93-6,O=C1CCCCC(=O)N1,0.750180,22586.0000
...,...,...,...,...
1924,3385-03-3,CC1(C)OC2CC3C4CC(F)C5=CC(=O)C=CC5(C)C4C(O)CC3(...,5.939000,0.4997
1925,2338-29-6,FC(F)(F)c1nc2c(Cl)c(Cl)c(Cl)c(Cl)c2[nH]1,6.121300,0.2435
1926,128606-48-4,CCOP(=S)(OCC)O/C(C)=C/C(=O)OC,6.282400,0.1399
1927,50585-41-6,Brc1cc2c(cc1Br)Oc1cc(Br)c(Br)cc1O2,6.698800,0.0992


 Convert a SMILES string to canonical SMILES

In [16]:
df_ts1 = deepcopy(df_ts)
df_ts1["SMILES"] = df_ts1.apply(lambda x: convert_smi_to_canon_smi(x.SMILES), axis=1)
df_ts1

,CAS_Number,SMILES,pLD50,"LD50,mg/kg"
0,7782-40-3,C,0.017765,15388.8000
1,2842-38-8,OCCNC1CCCCC1,0.572840,38274.0000
2,66257-53-2,NC(=O)C(=O)O,0.624490,21133.0000
3,2173-56-0,CCCCCOC(=O)CCCC,0.686960,35395.0000
4,4726-93-6,O=C1CCCCC(=O)N1,0.750180,22586.0000
...,...,...,...,...
1924,3385-03-3,CC1(C)OC2CC3C4CC(F)C5=CC(=O)C=CC5(C)C4C(O)CC3(...,5.939000,0.4997
1925,2338-29-6,FC(F)(F)c1nc2c(Cl)c(Cl)c(Cl)c(Cl)c2[nH]1,6.121300,0.2435
1926,128606-48-4,CCOP(=S)(OCC)OC(C)=CC(=O)OC,6.282400,0.1399
1927,50585-41-6,Brc1cc2c(cc1Br)Oc1cc(Br)c(Br)cc1O2,6.698800,0.0992


In [17]:
print('Original data: ', len(df_ts), 'molecules')
print('Failed data: ', len(df_ts1[df_ts1['SMILES']=='wrong_smiles']), 'molecules')

Original data:  1929 molecules
Failed data:  0 molecules


In [18]:
index=df_ts1.index[df_ts1['SMILES']=='wrong_smiles'].tolist()
wrong_smiles=df_ts.iloc[index]
wrong_smiles=wrong_smiles.SMILES
number=[x+1 for x in index]
bad_molecules = pd.DataFrame({'No. failed smiles in original set': number, 'SMILES of wrong structure: ': wrong_smiles}, index=None)
bad_molecules = bad_molecules.set_index('No. failed smiles in original set')
bad_molecules

,SMILES of wrong structure:
No. failed smiles in original set,


##  Standardization  for test set

In [22]:
df_ts1["Molecule"] = df_ts1.apply(lambda x: standart(x.SMILES), axis=1)
moldf_ts=df_ts1[df_ts1['SMILES']!='wrong_smiles']
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  1929 molecules


In [23]:
moldf_ts

,CAS_Number,SMILES,pLD50,"LD50,mg/kg",Molecule
0,7782-40-3,C,0.017765,15388.8000,<rdkit.Chem.rdchem.Mol object at 0x000001C71BA...
1,2842-38-8,OCCNC1CCCCC1,0.572840,38274.0000,<rdkit.Chem.rdchem.Mol object at 0x000001C71BA...
2,66257-53-2,NC(=O)C(=O)O,0.624490,21133.0000,<rdkit.Chem.rdchem.Mol object at 0x000001C71FD...
3,2173-56-0,CCCCCOC(=O)CCCC,0.686960,35395.0000,<rdkit.Chem.rdchem.Mol object at 0x000001C71FD...
4,4726-93-6,O=C1CCCCC(=O)N1,0.750180,22586.0000,<rdkit.Chem.rdchem.Mol object at 0x000001C71FD...
...,...,...,...,...,...
1924,3385-03-3,CC1(C)OC2CC3C4CC(F)C5=CC(=O)C=CC5(C)C4C(O)CC3(...,5.939000,0.4997,<rdkit.Chem.rdchem.Mol object at 0x000001C71FD...
1925,2338-29-6,FC(F)(F)c1nc2c(Cl)c(Cl)c(Cl)c(Cl)c2[nH]1,6.121300,0.2435,<rdkit.Chem.rdchem.Mol object at 0x000001C71FD...
1926,128606-48-4,CCOP(=S)(OCC)OC(C)=CC(=O)OC,6.282400,0.1399,<rdkit.Chem.rdchem.Mol object at 0x000001C71FD...
1927,50585-41-6,Brc1cc2c(cc1Br)Oc1cc(Br)c(Br)cc1O2,6.698800,0.0992,<rdkit.Chem.rdchem.Mol object at 0x000001C71FD...


In [24]:
y_ts=moldf_ts.pLD50
y_ts

0       0.017765
1       0.572840
2       0.624490
3       0.686960
4       0.750180
          ...   
1924    5.939000
1925    6.121300
1926    6.282400
1927    6.698800
1928    9.541100
Name: pLD50, Length: 1929, dtype: float64

In [25]:
moldf_ts=moldf_ts.Molecule

## Calculation MorganFingerprint for work set

In [26]:
fp_tr = [AllChem.GetMorganFingerprintAsBitVect(m, radius=2,nBits=1024,useFeatures=False,useChirality = False) for m in moldf_ws]

In [27]:
def rdkit_numpy_convert(fp_tr):
    output = []
    for f in fp_tr:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [28]:
from numpy import savetxt
x_tr = rdkit_numpy_convert(fp_tr)

In [23]:
savetxt('Models/MorganFingerprint/x_tr.csv', x_tr, delimiter=',')

In [29]:
x_tr.shape

(7712, 1024)

## Calculation MorganFingerprint for test set

In [30]:
fp_ts = [AllChem.GetMorganFingerprintAsBitVect(m, radius=2,nBits=1024,useFeatures=False,useChirality = False) for m in moldf_ts]

In [31]:
def rdkit_numpy_convert(fp_ts):
    output = []
    for f in fp_ts:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [32]:
x_ts = rdkit_numpy_convert(fp_ts)

In [33]:
x_ts.shape

(1929, 1024)

In [34]:
x_tr = np.array(x_tr, dtype=np.float32)
y_tr = np.array(y_tr, dtype=np.float32)

# load the models from disk

In [36]:
best_svm = pickle.load(open('models/MorganFingerprint/Toxicity_SVM_MF.pkl', 'rb'))

In [37]:
best_gbr = pickle.load(open('models/MorganFingerprint/Toxicity_GBR_MF.pkl', 'rb'))

In [38]:
best_MLPR = pickle.load(open('models/MorganFingerprint/Toxicity_MLPR_MF.pkl', 'rb'))

# Prediction for CV

In [39]:
y_pred_ws_GBR = best_gbr.predict(x_tr)

In [40]:
y_pred_ws_svm = best_svm.predict(x_tr)

In [41]:
y_pred_con=(y_pred_ws_GBR+y_pred_ws_svm)/2

In [42]:
R2_WS = round(r2_score(y_tr, y_pred_con), 2)
R2_WS

0.9

In [43]:
RMSE_WS=round(np.sqrt(mean_absolute_error(y_tr, y_pred_con)), 2)
RMSE_WS

0.46

In [44]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [45]:
y_pred_CV_svm = cross_val_predict(best_svm, x_tr, y_tr, cv=cv)

In [46]:
y_pred_CV_gbr = cross_val_predict(best_gbr, x_tr, y_tr, cv=cv)

In [47]:
y_pred_CV_MLPR = cross_val_predict(best_MLPR, x_tr, y_tr, cv=cv)

# three models: svm+gbr+MLPR

In [48]:
y_pred_con=(y_pred_CV_svm+y_pred_CV_gbr+y_pred_CV_MLPR)/3

In [49]:
Q2_CV = round(r2_score(y_tr, y_pred_con), 2)
Q2_CV

0.54

In [50]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_con)),2)
RMSE_CV

0.66

# two models: svm+gbr

In [51]:
y_pred_con_without_MLPR=(y_pred_CV_svm+y_pred_CV_gbr)/2

In [52]:
Q2_CV = round(r2_score(y_tr, y_pred_con_without_MLPR), 2)
Q2_CV

0.54

In [53]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_con_without_MLPR)),2)
RMSE_CV

0.66

# Prediction for test set's molecules

In [54]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [55]:
y_pred_svm = best_svm.predict(x_ts)

In [56]:
y_pred_gbr = best_gbr.predict(x_ts)

In [57]:
y_pred_MLPR = best_MLPR.predict(x_ts)

In [58]:
y_pred_con=(y_pred_svm+y_pred_gbr+y_pred_MLPR)/3

In [59]:
Q2_TS = round(r2_score(y_ts, y_pred_con), 2)
Q2_TS

0.58

In [60]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_con)), 2)
RMSE_TS

0.64

In [61]:
y_pred_con_without_MLPR=(y_pred_svm+y_pred_gbr)/2

In [62]:
Q2_TS = round(r2_score(y_ts, y_pred_con_without_MLPR), 2)
Q2_TS

0.57

In [63]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts,  y_pred_con_without_MLPR)), 2)
RMSE_TS

0.65

# Estimating applicability domain. Method - Euclidian distances, K=1

In [64]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [65]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,7702,7703,7704,7705,7706,7707,7708,7709,7710,7711
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.162278,4.000000,1.414214,3.741657,2.236068,3.464102,3.316625,6.000000,0.000000,2.000000,...,2.449490,3.316625,4.000000,4.795832,4.358899,2.645751,6.557438,3.316625,2.645751,5.567764
2,3.464102,4.690416,2.236068,3.872983,2.449490,3.464102,3.605551,6.082763,0.000000,2.449490,...,2.645751,3.464102,4.690416,5.099020,5.477226,4.472136,6.557438,4.242640,2.645751,5.916080
3,3.872983,4.795832,2.449490,3.872983,2.449490,3.464102,3.741657,6.082763,0.000000,2.449490,...,2.828427,4.000000,6.164414,6.164414,5.477226,5.099020,6.633250,5.099020,2.828427,6.244998
4,3.872983,5.099020,2.449490,4.000000,2.449490,3.464102,3.741657,6.082763,1.732051,2.645751,...,3.741657,4.242640,6.244998,6.782330,5.477226,5.099020,6.633250,7.416198,3.316625,6.324555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,10.630146,10.816654,10.583005,10.723805,10.246951,10.344080,10.677078,10.723805,10.677078,10.816654,...,10.954452,11.224972,11.224972,11.916375,11.357817,11.661903,11.224972,11.832160,10.677078,11.489125
7708,11.180340,11.045361,11.224972,11.445523,10.908712,11.090536,10.862781,11.224972,11.313708,11.357817,...,11.045361,11.747340,11.401754,12.206555,11.532562,11.958261,11.618950,12.247449,10.770329,11.832160
7709,11.704700,11.832160,11.747340,12.041595,11.357817,11.445523,11.401754,11.747340,11.747340,11.789826,...,12.000000,12.083046,11.832160,12.767145,12.124355,12.529964,12.206555,12.328828,11.747340,12.727922
7710,11.874342,11.916375,11.747340,12.124355,11.532562,11.532562,11.489125,11.916375,11.832160,11.958261,...,12.000000,12.165525,12.083046,13.000000,12.288206,12.688578,12.369317,12.569805,11.832160,12.884099


In [66]:
similarity= neighbors_k

In [67]:
Dmean=np.mean(similarity[1,:])

In [68]:
round(Dmean, 2)

3.63

In [69]:
std=np.std(similarity[1,:])

In [70]:
round(std, 2)

1.23

In [71]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

4.25


In [72]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [73]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928
0,2.000000,3.464102,2.000000,1.414214,2.000000,2.645751,3.162278,0.000000,2.449490,4.358899,...,6.855655,4.690416,2.828427,3.741657,3.162278,4.898980,2.645751,4.123106,3.162278,7.071068
1,2.000000,3.464102,2.236068,1.732051,3.162278,2.828427,3.605551,2.449490,2.449490,4.690416,...,7.071068,6.000000,3.316625,3.872983,3.162278,5.916080,2.828427,4.358899,3.605551,7.280110
2,2.000000,3.605551,2.236068,1.732051,3.162278,2.828427,3.872983,2.449490,2.449490,4.795832,...,7.280110,6.782330,3.316625,4.795832,3.162278,6.082763,2.828427,4.582576,3.741657,7.348469
3,2.000000,3.872983,2.645751,2.000000,3.464102,2.828427,4.000000,2.449490,2.449490,5.196152,...,7.280110,6.855655,3.464102,4.795832,3.316625,6.082763,2.828427,4.690416,3.741657,7.348469
4,2.000000,4.000000,2.828427,2.236068,3.605551,2.828427,4.242640,2.449490,2.645751,5.196152,...,7.416198,6.855655,3.605551,4.898980,3.605551,6.480741,3.000000,4.690416,3.872983,7.416198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,10.583005,10.583005,10.295630,10.770329,10.246951,10.344080,11.000000,10.488089,10.535654,10.954452,...,11.789826,11.618950,11.401754,11.180340,11.269427,11.789826,11.357817,11.224972,10.770329,11.489125
7708,11.135529,11.135529,10.954452,11.313708,11.045361,11.000000,11.618950,11.135529,11.000000,11.224972,...,12.124355,12.124355,11.661903,11.445523,11.618950,11.874342,11.704700,11.661903,10.954452,12.083046
7709,11.575837,11.661903,11.575837,11.832160,11.575837,11.532562,11.704700,11.661903,11.618950,11.747340,...,12.449900,12.609520,12.165525,12.124355,11.958261,12.206555,12.124355,12.083046,11.832160,12.165525
7710,11.661903,11.661903,11.575837,11.916375,11.661903,11.532562,11.789826,11.661903,11.704700,12.000000,...,12.845233,12.688578,12.247449,12.124355,12.041595,12.369317,12.206555,12.247449,11.832160,12.247449


In [74]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[2.    3.464 2.    ... 4.123 3.162 7.071]


In [75]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True ...  True  True False]


In [76]:
print("Coverage = ", sum(cpd_AD) / len(cpd_AD))

Coverage =  0.7407983411093831


In [77]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [   0    1    2 ... 1925 1926 1927]


In [78]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [79]:
y_pred_con_ad=list(y_pred_con)

In [80]:
y_pred_con_ad[:] = [x for i,x in enumerate(y_pred_con_ad) if i not in out_Ad]

In [81]:
len(y_pred_con_ad)

1429

In [82]:
y_ts_ad=list(y_ts)

In [83]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [84]:
len(y_ts_ad)

1429

In [85]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_con_ad), 2)
Q2_TS

0.66

In [86]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_con_ad)), 2)
RMSE_TS

0.63